In [ ]:
import numpy as np
import xarray as xr
import geoviews as gv
import datashader as dsh
from geoviews import opts

gv.extension('bokeh', 'matplotlib')

opts.defaults(
    opts.Image(width=600, height=400, colorbar=True),
    opts.Feature(apply_ranges=False),
    opts.QuadMesh(width=600, height=400, colorbar=True))

In geographical applications grids and meshes of different kinds are very common and for visualization and analysis it is often very important to be able to resample them in different ways. Regridding can refer both to upsampling and downsampling a grid or mesh, which is achieved through interpolation and aggregation.

Naive approaches to regridding treat the space as flat, which is often simpler but can also give less accurate results when working with a spherical space such as the earth. In this user guide we will summarize how to work with different grid types including rectilinear, curvilinear grids and trimeshes. Additionally we will discuss different approaches to regridding working based on the assumption of a flat earth (using [datashader](http://datashader.org/)) and a spherical earth ([xESMF](http://xesmf.readthedocs.io/en/latest/Curvilinear_grid.html)).

# Rectilinear grids

Rectilinear grids are one of the most standard formats and are defined by regularly sampled coordinates along the two axes. The ``air_temperature`` dataset provided by xarray and used throughout the GeoViews documentation provides a good example.

In [ ]:
ds = xr.tutorial.open_dataset('air_temperature').load().isel(time=slice(0, 100))
ds

As we already know from the [Gridded Dataset](./user_guide/Gridded_Datasets_II.ipynb) sections, an xarray of this kind can easily be wrapped in a GeoViews dataset:

In [ ]:
gvds = gv.Dataset(ds).redim.range(air=(230, 300))
gvds

We can also easily plot this data by using the ``.to`` method to group the data into a set of ``Image`` elements indexed by 'time':

In [ ]:
images = gvds.to(gv.Image, ['lon', 'lat'], dynamic=True)

It is important to note that if we look at the longitude coordinates above we can see that they are defined in the range (0, 360), while GeoViews generally expects it to be in the range (-180, 180). To correct this we can apply a simple fix:

In [ ]:
ds['lon'] = np.where(ds.lon>180, ds.lon-360, ds.lon)

Now we are ready to display the data. Note that throughout this user guide we will be using Bokeh but we could easily switch to matplotlib if needed.

In [ ]:
opts.defaults(opts.Image(cmap='viridis'))

images * gv.feature.coastline

### Datashader

HoloViews provides high-level wrappers around the datashader library, which make it possible to quickly resample or aggregate datasets of different kinds. Datashader knows nothing about non-flat coordinate systems, but provides a very fast, parallelized regridding operation for rectilinear grids. Here we will import the ``regrid`` operation and pass it our stack of images from above. While this dataset is fairly small and regridding will actually upsample the image to match the dimensions of the plot, ``regrid`` can very quickly downsample very large datasets.

One important thing to note about the resampling operations we will be working with in this user guide is that they are dynamic and linked to the plot dimensions and axis ranges. This means that whenever we zoom or pan the data will be resampled. If we want to disable this linked behavior and supply an explicit width and height we can disable the streams by passing ``streams=[]`` as a keyword argument.

In [ ]:
from holoviews.operation.datashader import regrid
regrid(images) * gv.feature.coastline

### xESMF

The xESMF library is specifically designed to provide an easy way to accurately resample grids defined in geographic coordinate systems and differs significantly from the simpler approach used by datashader, which applies simple upsampling and downsampling. xESMF is a wrapper around the [ESMF regridding algorithms](https://www.earthsystemcog.org/projects/esmf/regridding), which compute an interpolation weight matrix which is applied to remap the values of the source grid onto the destination grid. 

In GeoViews these algorithms are made available via the ``weighted_regrid`` operation, which supports the different interpolation modes including: 'bilinear', 'nearest_s2d', 'nearest_d2s' and 'conservative'. Since generating the sparse weight matrix takes much longer than applying it the operation will cache the weight matrix on disk for later use; this optimization can be disabled via the ``reuse_weights`` parameter or customized by defining a custom ``file_pattern``.

In [ ]:
from geoviews.operation.regrid import weighted_regrid
weighted_regrid(images) * gv.feature.coastline

Since this operation creates local weight files we will want to clean up after ourselves once we are done, to do so we can call the ``weighted_regrid.clean_weight_files`` method.

In [ ]:
weighted_regrid.clean_weight_files()

#### Onto an existing grid

The operation also allows us to define a target grid, which we can either define manually or by using a utility provided by the xESMF library. Here we will define a $2^{\circ}\times2^{\circ}$ grid.

In [ ]:
import xesmf as xe
grid = xe.util.grid_2d(-160, -35, 2, 15, 70, 2)
grid

Since the grid has 2D coordinate arrays the regridded data will be wrapped in and displayed as a QuadMesh:

In [ ]:
target = gv.Dataset(grid, kdims=['lon', 'lat'])
weighted_regrid(images, target=target, streams=[]) * gv.feature.coastline

# Curvilinear Grids

Curvilinear grids are another very common mesh type, which are usually defined by multi-dimensional coordinate arrays:

In [ ]:
ds = xr.tutorial.open_dataset('rasm').load()
ds

Just like the rectilinear grids GeoViews understands this kind of data natively. So we again wrap this dataset in a ``gv.Dataset`` and define a fixed range for the air teperature (``Tair``) values:

In [ ]:
gvds = gv.Dataset(ds).redim.range(Tair=(-25, 25))
gvds

Now we can plot this data directly as a ``gv.QuadMesh``, however this is generally quite slow, especially when we are working with bokeh where each grid point is rendered as a distinct polygon. We will therefore downsample the data by a factor of 3 along both dimensions:

In [ ]:
opts.defaults(opts.Image(cmap='RdBu_r'), opts.QuadMesh(cmap='RdBu_r'))

quadmeshes = gvds.to(gv.QuadMesh, ['xc', 'yc'], dynamic=True)
quadmeshes.map(lambda x: x.clone(x.data.Tair[::3, ::3]), gv.QuadMesh) * gv.feature.coastline

The problem is less severe when plotting using matplotlib but even then plotting can be fairly slow given a large enough mesh.

In [ ]:
gv.output(quadmeshes.opts(cmap='RdBu_r') * gv.feature.coastline, backend='matplotlib', size=300)

If we want to explore a very large grid it therefore often makes sense to resample the data onto a rectilinear grid, which can be rendered much more efficiently. Once again we have the option of using the datashader based approach or the more accurate xESMF based approach.

### Datashader

To regrid a ``QuadMesh`` using GeoViews we can import the ``rasterize`` operation. In the background the operation will convert the ``QuadMesh`` into a ``TriMesh``, which datashader understands. To optimize this conversion so it occurs only when aggregating the ``QuadMesh`` for the first time we can activate the ``precompute`` option. Additionally we have to define an aggregator, in this case to compute the mean ``Tair`` value in a pixel:

In [ ]:
from holoviews.operation.datashader import rasterize
rasterize(quadmeshes, precompute=True, aggregator=dsh.mean('Tair')) * gv.feature.coastline

### xESMF

Now we will once again use the xESMF based regridding for which we can still use the ``weighted_regrid`` operation, since it supports both rectilinear and curvilinear grids. Since the original data doesn't have a very high resolution we will also disable the ``streams`` linking the operation to the plot dimensions and axis ranges.

In [ ]:
from geoviews.operation.regrid import weighted_regrid
weighted_regrid(quadmeshes, streams=[]) * gv.feature.coastline

Finally lets clean up after ourselves one last time:

In [ ]:
weighted_regrid.clean_weight_files()